In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import random
import pickle

# figure(figsize=(20, 20), dpi=80)

dataset = pd.read_excel("./drive/MyDrive/Data1.xlsx")
# dataset.head()

In [3]:
class OilDataset(Dataset):
  def __init__(self, data):
    # data loading
    self.x = torch.from_numpy(data[:,:-1]).type(torch.FloatTensor)
    self.y = torch.from_numpy(data[:,-1].reshape(-1, 1)).type(torch.FloatTensor)

    self.n_samples = data.shape[0]

  def __getitem__(self, index):
    # dataset index 0
    return self.x[index], self.y[index]

  def __len__(self):
    # cll alen of dataset
    return self.n_samples


In [4]:
class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()

    self.linear1 = nn.Linear(n_input_features, 1)
    # self.linear2 = nn.Linear(3, 1)

  def forward(self, x):
    y_pred = self.linear1(x)
    # y_pred = self.linear2(y_pred)
    return y_pred

  

In [5]:
from math import gamma
class TrendAnalizer():
  def __init__(self, dataset):
    self.scale_data(dataset)

  def train(self, model, train_loader, epochs):
    print('Please Wait... ')
    l_rate = 0.01
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=l_rate)

    epochs = epochs
    steps = len(train_loader)
    device = torch.device('cpu')

    for epoch in range(epochs):
      for i, (x, y) in enumerate(train_loader):
        inputs = x.to(device)
        train_d = x.to(device)
        labels = y.to(device)
        train_yd = y.to(device)
        
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      if (epoch+1) % epochs == 0:
        print(f'epoch: {epoch+1} / {epochs}, loss: {loss.item():.4f}')

  def test(self, model, OilUS_test_dataset):
    with torch.no_grad():
      lenn = len(OilUS_test_dataset.x)
      count = 0
      pred = []
      for inp in OilUS_test_dataset.x:
        count += 1
        out = model(inp)
        pd = list(np.array(out.detach()).reshape(-1))

        pred += pd

        if count == lenn:
          new_inp = inp
          for i in range(12):
            new_inp = new_inp[1:]
            new_inp = torch.tensor(list(np.array(new_inp.detach())) + pd)
            # print(new_inp)
            # g

            out = model(new_inp)
            pd = list(np.array(out.detach()).reshape(-1))
            # pd[0] = pd[0] + g
            # print(pd[0])
            # print(scaler)

            pred += pd
      return pred


  def structureData(self, dataset):
    dataset = dataset.reshape(len(dataset),)
    ln = len(dataset)
    i = 0
    j = 6
    counter_array = []
    total_data = []
    s = True
    
    while s:
      for entry in dataset[i:j]:
        counter_array.append(entry)
        
      if j == ln:
        s = False
      else:
        i += 1
        j += 1

      total_data.append(counter_array)
      counter_array = []
    return np.array(total_data)

    
  def scale_data(self, dataset):
    col = dataset.columns[1:]
    list_of_col_data = {}

    for i in col:
      sc = StandardScaler()
      c = np.array(dataset[i], dtype='float32').reshape(-1, 1)
      c = sc.fit_transform(c)

      list_of_col_data[i] = np.array([sc, c])

    self.dataset = list_of_col_data

In [6]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def plot(col_name, prediction, cut, all_pip, scaler):
  new_prediction = []
  for pred in prediction[-13:]:
    g = random.randrange(-200, 200)
    prd = np.array(pred).reshape(-1, 1)
    print(prd)
    # g = scaler.transform(g)
    prd = scaler.inverse_transform(prd)
    print(prd)
    # g = g[0][0]
    print('random')
    print(g)
    nw = prd[0][0] + g
    print("Addition: ")
    print(nw)
    print('addition to scale: ')
    neww = scaler.transform(np.array(nw).reshape(-1, 1))
    print(neww)
    # print(pred)
    # print("+")
    # print(g)
    # print(nw)
    new_prediction.append(neww[0][0])
  
  # prediction = prediction[:-12] + new_prediction


  deed1 = np.array(dataset[col_name][:-3])[cut:]
  deed1 = np.array(deed1)
  last_date = deed1[-1]

  deed = [last_date,]

  # print(type(deed))
  for i in range(12):
    date_after_month = last_date + relativedelta(months=1)
    deed = np.append(deed, [date_after_month,])
    last_date = date_after_month

  plt.figure(figsize=(15,6))

  
  plt.figure(figsize=(15,6))
  plt.plot(np.array(dataset[col_name][:-3])[:cut], all_pip[:cut], 'g')
  plt.plot(np.array(dataset[col_name][:-3])[cut:], all_pip[cut:], 'b')
  plt.plot(deed1, np.array(prediction[:-12]), 'r')
  plt.plot(deed, np.array(new_prediction), 'y')
  
  plt.show()

def plt_exact(col_name, predicted, cut, scaler):
  new_prediction = []
  for pred in predicted[-12:]:
    g = random.randrange(-200, 200)
    g = np.array(g).reshape(-1, 1)
    g = scaler.transform(g)
    g = g[0][0]
    nw = pred + g
    new_prediction.append(nw)

  # predicted = predicted[:-12] + new_prediction

  deed1 = np.array(dataset[col_name][:-3])[cut:]
  deed1 = np.array(deed1)
  last_date = deed1[-1]

  deed = []

  # print(type(deed))
  for i in range(12):
    date_after_month = last_date + relativedelta(months=1)
    deed = np.append(deed, [date_after_month,])
    last_date = date_after_month

  plt.figure(figsize=(15,6))

  plt.figure(figsize=(15,6))
  plt.plot(np.array(dataset[col_name][:-3])[cut:], all_pip[cut:], 'b')
  plt.plot(deed1, np.array(predicted[:-12]), 'r')
  plt.plot(deed, np.array(new_prediction), 'y')

  plt.show()

In [ ]:

trend = TrendAnalizer(dataset[:-3])

In [ ]:
config = {}

for cols in dataset.columns[1:]:
  data1 = None
  scaler = None
  
  model = LogisticRegression(5)

  data1 = trend.dataset[cols]

  scaler = data1[0]

  data_structured = trend.structureData(data1[1])

  train_data_dataset = OilDataset(data_structured[:-80])
  test_data_dataset = OilDataset(data_structured[-80:])

  train_loader = DataLoader(dataset=train_data_dataset, batch_size=5, shuffle=False, num_workers=2)

  trend.train(model, train_loader, 60)
  
  test_loader = DataLoader(dataset=test_data_dataset, batch_size=5, shuffle=False, num_workers=2)

  predicted = trend.test(model, test_data_dataset)

  all_pip = np.array(np.array(data1[1]).reshape(len(data1[1],)))

  new_prediction = []

  for pred in predicted[-13:]:
    g = random.randrange(-200, 200)
    prd = np.array(pred).reshape(-1, 1)
    prd = scaler.inverse_transform(prd)
    nw = prd[0][0] + g
    neww = scaler.transform(np.array(nw).reshape(-1, 1))
    
    new_prediction.append(neww[0][0])
    


  deed1 = np.array(dataset['Date'][:-3])[-80:]
  deed1 = np.array(deed1)
  last_date = deed1[-1]

  deed = [last_date,]
  
  for i in range(12):
    date_after_month = last_date + relativedelta(months=1)
    deed = np.append(deed, [date_after_month,])
    last_date = date_after_month

  plot_data = {
    'x01':np.array(dataset[cols][:-3])[:-80],
    'y01':all_pip[:-80],

    'x02':np.array(dataset[cols][:-3])[-80:],
    'y02':all_pip[-80:],

    'x03':deed1,
    'y03':np.array(predicted[:-12]),

    'x04':deed,
    'y04':np.array(new_prediction)
  }

  config[cols.replace(" ", "")] = {plot_data}
  torch.save(model.state_dict(), cols.replace(" ", "")+".pth")
  
  plot('Date', 
      predicted,
      -80, 
      all_pip,
      scaler
  )

  plt_exact('Date', 
      predicted,
      -80,
      scaler
  )


In [28]:
configs = pd.DataFrame(config).transpose()

dat = dict(configs['plot_data'])



In [29]:
pickle.dump(dat, open('config.pickle', 'wb'))